In [ ]:
from threaded_batching import ThreadedBatcher

In [ ]:
class TestBatcher(ThreadedBatcher):
    
    def __init__(self, num_batches=100, **kwargs):
        self._next_batch_idx = 0
        self._num_batches = 100
        super(TestBatcher, self).__init__(**kwargs)
    
    def _get_next_batch_params(self):
        if self._next_batch_idx < self._num_batches:
            self._next_batch_idx += 1
            return (self._next_batch_idx-1)
        else:
            return None
    
    def _get_batch_from_params(self, params):
        return params

In [ ]:
tds = TestBatcher()

In [ ]:
for batch in tds:
    print(batch)